In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *

In [2]:
from pyspark.sql import Row

In [3]:
spark = SparkSession.builder.getOrCreate()

22/10/30 14:59:35 WARN Utils: Your hostname, wangjiangyideMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
22/10/30 14:59:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/30 14:59:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
import findspark
findspark.init()

In [5]:
from os import environ
environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.15.0 pyspark-shell'

In [6]:
from pyspark import find_spark_home
print(find_spark_home._find_spark_home())

/Users/wangjiangyi/miniforge3/lib/python3.8/site-packages/pyspark


# Import MAS DB

In [7]:
df_author = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", "author") \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


In [8]:
df_writes = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", "writes") \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

In [9]:
df_publication = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", "publication") \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

In [10]:
df_author.printSchema()

root
 |-- aid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- oid: integer (nullable = true)
 |-- homepage: string (nullable = true)
 |-- photo: string (nullable = true)



In [11]:
df_writes.printSchema()

root
 |-- aid: integer (nullable = true)
 |-- pid: integer (nullable = true)



In [12]:
df_publication.printSchema()

root
 |-- pid: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- cid: integer (nullable = true)
 |-- jid: integer (nullable = true)
 |-- reference_num: integer (nullable = true)
 |-- citation_num: integer (nullable = true)
 |-- doi: string (nullable = true)



# Import DBLP DB

In [13]:
# define schema
article_schema = StructType([
    StructField("author", ArrayType(elementType = StringType()), True),
    StructField("title", StringType(), True),
    StructField("year", LongType(), True),
    StructField("journal", StringType(), True)])

In [14]:
df_article = spark.read \
     .format('xml') \
     .option('rootTag', 'dblp') \
     .option('rowTag', 'article') \
     .load("dblp.xml", schema = article_schema)

In [15]:
df_article.printSchema()

root
 |-- author: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)
 |-- journal: string (nullable = true)



In [16]:
df_article.show(5)

+--------------------+--------------------+----+--------------------+
|              author|               title|year|             journal|
+--------------------+--------------------+----+--------------------+
|[Paul Kocher, Dan...|Spectre Attacks: ...|2018|  meltdownattack.com|
|[Moritz Lipp, Mic...|            Meltdown|2018|  meltdownattack.com|
|      [Frank Manola]|An Evaluation of ...|1994|GTE Laboratories ...|
|[Michael L. Brodi...|DARWIN: On the In...|1993|GTE Laboratories ...|
|[Mark F. Hornick,...|Integrating Heter...|1991|GTE Laboratories ...|
+--------------------+--------------------+----+--------------------+
only showing top 5 rows



In [17]:
# define schema
inproc_schema = StructType([
    StructField("author", ArrayType(elementType = StringType()), True),
    StructField("title", StringType(), True),
    StructField("year", LongType(), True),
    StructField("booktitle", StringType(), True)])

In [18]:
df_inproc = spark.read \
     .format('xml') \
     .option('rootTag', 'dblp') \
     .option('rowTag', 'inproceedings') \
     .load("dblp.xml", schema = inproc_schema)

In [19]:
df_inproc.printSchema()

root
 |-- author: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)
 |-- booktitle: string (nullable = true)



In [20]:
df_inproc.show(5)

+--------------------+--------------------+----+--------------------+
|              author|               title|year|           booktitle|
+--------------------+--------------------+----+--------------------+
|   [Arnon Rosenthal]|The Future of Cla...|1998|                SWEE|
|        [E. F. Codd]|Seven Steps to Re...|1974|IFIP Working Conf...|
|[Toni Bollinger, ...|The LILOG Inferen...|1991|Text Understandin...|
|[Kai-Uwe Carstens...|Why a Hill Can't ...|1991|Text Understandin...|
|[David W. Flater,...|Towards Flexible ...|1993|Advanced Database...|
+--------------------+--------------------+----+--------------------+
only showing top 5 rows



### rename for article schema
- schema format: author - title - year - booktitle

In [21]:
df_article = df_article.withColumnRenamed('journal', 'booktitle')

In [22]:
df_article.show(5)

+--------------------+--------------------+----+--------------------+
|              author|               title|year|           booktitle|
+--------------------+--------------------+----+--------------------+
|[Paul Kocher, Dan...|Spectre Attacks: ...|2018|  meltdownattack.com|
|[Moritz Lipp, Mic...|            Meltdown|2018|  meltdownattack.com|
|      [Frank Manola]|An Evaluation of ...|1994|GTE Laboratories ...|
|[Michael L. Brodi...|DARWIN: On the In...|1993|GTE Laboratories ...|
|[Mark F. Hornick,...|Integrating Heter...|1991|GTE Laboratories ...|
+--------------------+--------------------+----+--------------------+
only showing top 5 rows



# Query 1:

## 1. Max_year in MAS DB
- when I run spark.sql, it will always suffer from time limit issues
- so I can only pre-process the data and load to Jupyter Notebook

- this cannot work!!!

In [23]:
df_author.createOrReplaceTempView('author')
df_writes.createOrReplaceTempView('writes')
df_publication.createOrReplaceTempView('publication')

In [24]:
maxyear_MAS = spark.sql("select max(year) as max_year from publication\
                    where pid in \
                    (select pid from writes \
                    where aid = (select aid from author where name = 'Divesh Srivastava'))")

- this works

In [25]:
query_maxyear = "(select max(year) as max_year from publication\
                where pid in \
                (select pid from writes \
                where aid = (select aid from author where name = 'Divesh Srivastava'))\
                ) tmp"

In [26]:
maxyear_MAS = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", query_maxyear) \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

In [27]:
maxyear_MAS.show()

+--------+
|max_year|
+--------+
|    2012|
+--------+



In [28]:
# achieve the max_year element
max_year = maxyear_MAS.collect()
max_year[0]['max_year']

2012

## 2. Find all papers from DBLP that are published after 'Max_year' and written by 'Divesh S.'

In [29]:
from pyspark.sql.functions import array_contains

In [30]:
article_DS = df_article.filter((df_article.year > max_year[0]['max_year']) & (array_contains(df_article.author, 'Divesh Srivastava')))

In [31]:
article_DS.show()

+--------------------+--------------------+----+--------------------+
|              author|               title|year|           booktitle|
+--------------------+--------------------+----+--------------------+
|[Leon Bornemann, ...|Data Change Explo...|2018|  Datenbank-Spektrum|
|[Philip E. Brown,...|From Rocks to Peb...|2019|ACM Trans. Spatia...|
|[Donatella Firman...|Robust Entity Res...|2018|IEEE Data Eng. Bull.|
|[Tamraparni Dasu,...|Data Quality for ...|2016|IEEE Data Eng. Bull.|
|[Luciano Barbosa,...|Big Data Integrat...|2018|IEEE Data Eng. Bull.|
|[Shazia W. Sadiq,...|Letter from the S...|2016|IEEE Data Eng. Bull.|
|[Kareem El Gebaly...| Explanation Tables.|2018|IEEE Data Eng. Bull.|
| [Divesh Srivastava]|Letter from the I...|2021|IEEE Data Eng. Bull.|
|[Divesh Srivastav...|Ensuring High-Qua...|2019|ACM J. Data Inf. ...|
|[Aline Bessa, Jul...|Effective Discove...|2020|    Trans. Data Sci.|
|[Abolfazl Asudeh,...|Efficient Signal ...|2019|         SIGMOD Rec.|
|[Shazia Wasim Sad..

In [32]:
inproc_DS = df_inproc.filter((df_inproc.year > max_year[0]['max_year']) & (array_contains(df_inproc.author, 'Divesh Srivastava')))

In [33]:
inproc_DS.show()

+--------------------+--------------------+----+---------+
|              author|               title|year|booktitle|
+--------------------+--------------------+----+---------+
|[Graham Cormode, ...|UMicS: from anony...|2013|     CIKM|
|[Divesh Srivastav...|Exploring and Ana...|2022|     CIKM|
|[Xi He 0001, Ashw...|Composing Differe...|2017|      CCS|
|[Philip E. Brown,...|     Don't Cry Wolf.|2019|     DSAA|
|[Mehdi Kargar, Lu...|Effective Keyword...|2021|     ICDE|
|[Xian Li, Xin Lun...|Scaling up copy d...|2015|     ICDE|
|[Alexandar Mihayl...|FASTOD: Bringing ...|2018|     ICDE|
|[Trong Duc Nguyen...|Random Sampling f...|2020|     ICDE|
|[Alexandra Kim, L...|Summarizing Hiera...|2020|     ICDE|
|[Guoyao Feng, Luk...|Scalable Informat...|2017|     ICDE|
|[Leon Bornemann, ...|Structured Object...|2021|     ICDE|
|[Xin Luna Dong, D...|Big data integrat...|2013|     ICDE|
|[Hongyun Cai 0001...|Indexing evolving...|2016|     ICDE|
|[Pei Li, Jessica ...|abcOD: Mining Ban...|2022|     ICD

In [34]:
query1_result = article_DS.select('title', 'year').union(inproc_DS.select('title', 'year'))

In [35]:
query1_result.show()

+--------------------+----+
|               title|year|
+--------------------+----+
|Data Change Explo...|2018|
|From Rocks to Peb...|2019|
|Robust Entity Res...|2018|
|Data Quality for ...|2016|
|Big Data Integrat...|2018|
|Letter from the S...|2016|
| Explanation Tables.|2018|
|Letter from the I...|2021|
|Ensuring High-Qua...|2019|
|Effective Discove...|2020|
|Efficient Signal ...|2019|
|Data Quality: The...|2017|
|Effective Keyword...|2022|
|Constrained Priva...|2021|
|Discovering Conse...|2014|
|Indexing Evolving...|2015|
|Efficient and Eff...|2020|
|Alaska: A Flexibl...|2021|
|Truth Finding on ...|2015|
|Errata Note: Disc...|2019|
+--------------------+----+
only showing top 20 rows



In [36]:
query1_result.count()

157

- for store the result

In [37]:
query1_result.show(200, truncate = False)

+------------------------------------------------------------------------------------------------------------------+----+
|title                                                                                                             |year|
+------------------------------------------------------------------------------------------------------------------+----+
|Data Change Exploration Using Time Series Clustering.                                                             |2018|
|From Rocks to Pebbles: Smoothing Spatiotemporal Data Streams in an Overlay of Sensors.                            |2019|
|Robust Entity Resolution Using a CrowdOracle.                                                                     |2018|
|Data Quality for Temporal Streams.                                                                                |2016|
|Big Data Integration for Product Specifications.                                                                  |2018|
|Letter from the Special

# Data pre-processing for Query 2 and Query 3:
- After doing analysis, it is more convenient to define a pipeline to preprocess the data
- The pipeline is used to **combine** the data from MAS and DBLP that:
    - 1. written by "D. S."
    - 2. drop duplicates only for those which appear in both MAS and DBLP
- The schema for the output of pipeline is:
    - author(array) - title - year - booktitle_name - booktitle_fullname

## 1. Read MAS data as the form we need, compact authors to array and filter for "D. S."
- read as:
    - MAS_article, which contains all journals in MAS;
    - MAS_inproc, which contains all conferences in MAS;
- the schema is:
    - author - title - year - booktitle_name - booktitle_fullname

- read article from MAS (journal)

In [38]:
query_MAS_article = "(with write_array as ( \
select writes.pid as pid, GROUP_CONCAT(author.name) as name_list \
from writes left join author on author.aid = writes.aid \
group by writes.pid ) \
select \
	b.name_list 		as author, \
    a.title				as title, \
    a.year				as year, \
    c.name 				as booktitle_name, \
    c.full_name 		as booktitle_fullname \
from (select pid, title, year, jid from publication where jid != 0 and pid in (select pid from writes where aid = (select aid from author where name = 'Divesh Srivastava'))) a \
left join write_array b on a.pid = b.pid \
left join journal c on a.jid = c.jid \
) tmp"

In [39]:
MAS_article = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", query_MAS_article) \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

In [40]:
MAS_article.show()

+--------------------+--------------------+----+--------------+--------------------+
|              author|               title|year|booktitle_name|  booktitle_fullname|
+--------------------+--------------------+----+--------------+--------------------+
|Divesh Srivastava...|Using q-grams in ...|2001|          DEBU|IEEE Data(base) E...|
|Andrew Nierman,Di...|TIMBER: A native ...|2002|          VLDB|    The Vldb Journal|
|Divesh Srivastava...|Minimization of T...|2001| Sigmod Record|                    |
|Divesh Srivastava...|One-dimensional a...|2000|          VLDB|    The Vldb Journal|
|Divesh Srivastava...|Implementation of...|1993| Sigmod Record|                    |
|Divesh Srivastava...|On Effective Mult...|2000| Sigmod Record|                    |
|Dongwon Lee,Dives...|Generating Advanc...|1998|            CN|Computer Networks...|
|Divesh Srivastava...|Querying Network ...|1999| Sigmod Record|                    |
|Divesh Srivastava...|The CORAL Deducti...|1994|          VLDB|  

In [41]:
# convert string to array
from pyspark.sql.functions import col, split
MAS_article = MAS_article.withColumn("author", split(col("author"), ",").cast("array<string>"))

In [42]:
MAS_article.show()

+--------------------+--------------------+----+--------------+--------------------+
|              author|               title|year|booktitle_name|  booktitle_fullname|
+--------------------+--------------------+----+--------------+--------------------+
|[Divesh Srivastav...|Using q-grams in ...|2001|          DEBU|IEEE Data(base) E...|
|[Andrew Nierman, ...|TIMBER: A native ...|2002|          VLDB|    The Vldb Journal|
|[Divesh Srivastav...|Minimization of T...|2001| Sigmod Record|                    |
|[Divesh Srivastav...|One-dimensional a...|2000|          VLDB|    The Vldb Journal|
|[Divesh Srivastav...|Implementation of...|1993| Sigmod Record|                    |
|[Divesh Srivastav...|On Effective Mult...|2000| Sigmod Record|                    |
|[Dongwon Lee, Div...|Generating Advanc...|1998|            CN|Computer Networks...|
|[Divesh Srivastav...|Querying Network ...|1999| Sigmod Record|                    |
|[Divesh Srivastav...|The CORAL Deducti...|1994|          VLDB|  

In [43]:
MAS_article.printSchema()

root
 |-- author: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- booktitle_name: string (nullable = true)
 |-- booktitle_fullname: string (nullable = true)



- read inproceeding from MAS (conference)

In [44]:
query_MAS_inproc = "(with write_array as ( \
select writes.pid as pid, GROUP_CONCAT(author.name) as name_list \
from writes left join author on author.aid = writes.aid \
group by writes.pid ) \
select \
	b.name_list 		as author, \
    a.title				as title, \
    a.year				as year, \
    c.name 				as booktitle_name, \
    c.full_name 		as booktitle_fullname \
from (select pid, title, year, cid from publication where cid != 0 and pid in (select pid from writes where aid = (select aid from author where name = 'Divesh Srivastava'))) a \
left join write_array b on a.pid = b.pid \
left join conference c on a.cid = c.cid \
) tmp"

In [45]:
MAS_inproc = spark.read \
    .format('jdbc') \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("url", "jdbc:mysql://localhost:3306/test") \
    .option("dbtable", query_MAS_inproc) \
    .option("user", "root") \
    .option("password", "maomao2011") \
    .load()

In [46]:
# convert string to array
MAS_inproc = MAS_inproc.withColumn("author", split(col("author"), ",").cast("array<string>"))

In [47]:
MAS_inproc.show()

+--------------------+--------------------+----+--------------+--------------------+
|              author|               title|year|booktitle_name|  booktitle_fullname|
+--------------------+--------------------+----+--------------+--------------------+
|[Divesh Srivastav...|PIX: exact and ap...|2003|        SIGMOD|International Con...|
|[Divesh Srivastav...|Text Joins for Da...|2003|          ICDE|International Con...|
|[Nicolas Bruno, D...|Navigation vs. In...|2003|          ICDE|International Con...|
|[Divesh Srivastav...|Text joins in an ...|2003|           WWW|World Wide Web Co...|
|[Divesh Srivastav...| Ranked Join Indices|2003|          ICDE|International Con...|
|[Andrew Nierman, ...|TIMBER: a native ...|2003|        SIGMOD|International Con...|
|[Andrew Nierman, ...|     Grouping in XML|2002|EDBT(Workshop)|International Con...|
|[Divesh Srivastav...|Optimizing the se...|2002|          VLDB|Very Large Data B...|
|[Divesh Srivastav...|TAX: A Tree Algeb...|2001|          DBPL|Wo

In [48]:
MAS_inproc.printSchema()

root
 |-- author: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- booktitle_name: string (nullable = true)
 |-- booktitle_fullname: string (nullable = true)



## 2. As for DBLP data, filter for the array that contains "D. S.". Then, change the schema for dropping duplicates.
- the schema for DBLP is also:
    - author - title - year - booktitle_name - booktitle_fullname
    - booktitle_name and booktitle_fullname are the replicates of booktitle

In [49]:
from pyspark.sql.functions import regexp_replace
# copy publication_name to publication_full_name
df_article = df_article.withColumn('booktitle_fullname', regexp_replace('booktitle', '', ''))
df_inproc = df_inproc.withColumn('booktitle_fullname', regexp_replace('booktitle', '', ''))
df_article = df_article.withColumnRenamed('booktitle', 'booktitle_name')
df_inproc = df_inproc.withColumnRenamed('booktitle', 'booktitle_name')

In [50]:
DBLP_article = df_article.filter(array_contains(df_article.author, 'Divesh Srivastava'))
DBLP_inproc = df_inproc.filter(array_contains(df_inproc.author, 'Divesh Srivastava'))

In [51]:
DBLP_article.show()

+--------------------+--------------------+----+--------------------+--------------------+
|              author|               title|year|      booktitle_name|  booktitle_fullname|
+--------------------+--------------------+----+--------------------+--------------------+
|[Leon Bornemann, ...|Data Change Explo...|2018|  Datenbank-Spektrum|  Datenbank-Spektrum|
|[Divesh Srivastav...|Pushing Constrain...|1993|    J. Log. Program.|    J. Log. Program.|
|[Philip E. Brown,...|From Rocks to Peb...|2019|ACM Trans. Spatia...|ACM Trans. Spatia...|
|[Donatella Firman...|Robust Entity Res...|2018|IEEE Data Eng. Bull.|IEEE Data Eng. Bull.|
|[Luis Gravano, Pa...|Using q-grams in ...|2001|IEEE Data Eng. Bull.|IEEE Data Eng. Bull.|
|[Tamraparni Dasu,...|Data Quality for ...|2016|IEEE Data Eng. Bull.|IEEE Data Eng. Bull.|
|[Lukasz Golab, Fl...|Efficient and Eff...|2011|IEEE Data Eng. Bull.|IEEE Data Eng. Bull.|
|[Luciano Barbosa,...|Big Data Integrat...|2018|IEEE Data Eng. Bull.|IEEE Data Eng. Bull.|

In [52]:
DBLP_inproc.show()

+--------------------+--------------------+----+-------------------+-------------------+
|              author|               title|year|     booktitle_name| booktitle_fullname|
+--------------------+--------------------+----+-------------------+-------------------+
|[Raghu Ramakrishn...|Efficient Increme...|1994|               ILPS|               ILPS|
|[David B. Kemp, D...|Magic Sets and Bo...|1991|               ISLP|               ISLP|
|[S. Sudarshan 000...|Extending the Wel...|1993|               ILPS|               ILPS|
|[Nick Koudas, Div...|Data Stream Query...|2003|               WISE|               WISE|
|[H. V. Jagadish, ...|Flexible List Man...|1998|               CIKM|               CIKM|
|[Guy Jacobson, Ba...|Focusing Search i...|1998|               CIKM|               CIKM|
|[Graham Cormode, ...|UMicS: from anony...|2013|               CIKM|               CIKM|
|[Sihem Amer-Yahia...|Logical and physi...|2002|               CIKM|               CIKM|
| [Divesh Srivastava]

## 3. Manipulate for article and inproceedings respectively, i.e., union the MAS and DBLP data and drop duplicates.
- **add 2 new columns to store the REGULAR form of booktitle_name/booktitle_fullname**
- **add 1 new column to store the REGULAR form of title**
- we drop duplicates for booktitle_name and booktitle_fullname sequentially
    - to guarantee the duplicates between MAS and DBLP should be removed totally

### union MAS_article and DBLP_article, MAS_inproc and DBLP_inproc
    - rename as article, inproc correspondingly

In [53]:
article = DBLP_article.unionAll(MAS_article)
inproc = DBLP_inproc.unionAll(MAS_inproc)

In [54]:
article.first()

Row(author=['Leon Bornemann', 'Dmitri V. Kalashnikov', 'Felix Naumann', 'Divesh Srivastava'], title='Data Change Exploration Using Time Series Clustering.', year=2018, booktitle_name='Datenbank-Spektrum', booktitle_fullname='Datenbank-Spektrum')

In [55]:
inproc.first()

Row(author=['Raghu Ramakrishnan', 'Kenneth A. Ross', 'Divesh Srivastava', 'S. Sudarshan 0001'], title='Efficient Incremental Evaluation of Queries with Aggregation.', year=1994, booktitle_name='ILPS', booktitle_fullname='ILPS')

### create 3 new columns to store the REGULAR information

In [56]:
from pyspark.sql.functions import lower

    - title_processed

In [57]:
# Lower cases, drop blanks, drop dots
article = article.withColumn('title_processed', lower(col('title')))
article = article.withColumn('title_processed', regexp_replace('title_processed', ' ', ''))
article = article.withColumn('title_processed', regexp_replace('title_processed', r'\.', ''))


inproc = inproc.withColumn('title_processed', lower(col('title')))
inproc = inproc.withColumn('title_processed', regexp_replace('title_processed', ' ', ''))
inproc = inproc.withColumn('title_processed', regexp_replace('title_processed', r'\.', ''))

    - booktitle_name_processed

In [58]:
# Lower cases, drop blanks, drop dots
article = article.withColumn('booktitle_name_processed', lower(col('booktitle_name')))
article = article.withColumn('booktitle_name_processed', regexp_replace('booktitle_name_processed', ' ', ''))
article = article.withColumn('booktitle_name_processed', regexp_replace('booktitle_name_processed', r'\.', ''))


inproc = inproc.withColumn('booktitle_name_processed', lower(col('booktitle_name')))
inproc = inproc.withColumn('booktitle_name_processed', regexp_replace('booktitle_name_processed', ' ', ''))
inproc = inproc.withColumn('booktitle_name_processed', regexp_replace('booktitle_name_processed', r'\.', ''))

    - booktitle_fullname_processed

In [59]:
# Lower cases, drop blanks, drop dots
article = article.withColumn('booktitle_fullname_processed', lower(col('booktitle_fullname')))
article = article.withColumn('booktitle_fullname_processed', regexp_replace('booktitle_fullname_processed', ' ', ''))
article = article.withColumn('booktitle_fullname_processed', regexp_replace('booktitle_fullname_processed', r'\.', ''))


inproc = inproc.withColumn('booktitle_fullname_processed', lower(col('booktitle_fullname')))
inproc = inproc.withColumn('booktitle_fullname_processed', regexp_replace('booktitle_fullname_processed', ' ', ''))
inproc = inproc.withColumn('booktitle_fullname_processed', regexp_replace('booktitle_fullname_processed', r'\.', ''))

In [60]:
article.first()

Row(author=['Leon Bornemann', 'Dmitri V. Kalashnikov', 'Felix Naumann', 'Divesh Srivastava'], title='Data Change Exploration Using Time Series Clustering.', year=2018, booktitle_name='Datenbank-Spektrum', booktitle_fullname='Datenbank-Spektrum', title_processed='datachangeexplorationusingtimeseriesclustering', booktitle_name_processed='datenbank-spektrum', booktitle_fullname_processed='datenbank-spektrum')

In [61]:
inproc.first()

Row(author=['Raghu Ramakrishnan', 'Kenneth A. Ross', 'Divesh Srivastava', 'S. Sudarshan 0001'], title='Efficient Incremental Evaluation of Queries with Aggregation.', year=1994, booktitle_name='ILPS', booktitle_fullname='ILPS', title_processed='efficientincrementalevaluationofquerieswithaggregation', booktitle_name_processed='ilps', booktitle_fullname_processed='ilps')

### drop duplicates for article, inproc **correspondingly**
    - firstly, [year, title_processed, booktitle_name_processed] 
    - then, [year, title_processed, booktitle_fullname_processed]

- before dropping duplicates

In [62]:
article.count()

242

In [63]:
inproc.count()

422

- drop duplicates

In [64]:
article_dropdup = article.dropDuplicates(['year', 'title_processed', 'booktitle_name_processed'])
article_dropdup = article_dropdup.dropDuplicates(['year', 'title_processed', 'booktitle_fullname_processed'])

inproc_dropdup = inproc.dropDuplicates(['year', 'title_processed', 'booktitle_name_processed'])
inproc_dropdup = inproc_dropdup.dropDuplicates(['year', 'title_processed', 'booktitle_fullname_processed'])

- after dropping duplicates

In [65]:
article_dropdup.count()

228

In [66]:
inproc_dropdup.count()

321

# 4. Dealing with 'CoRR' case
- for each publication, if:
    - it is just published in 'CoRR', then preserve it;
    - it is published in 'CoRR' and other sources, then kick out it.

- article situation (there are many publications that are in 'CoRR')

In [67]:
article_dropdup.filter(article_dropdup.booktitle_name == 'CoRR').show()

+--------------------+--------------------+----+--------------+------------------+--------------------+------------------------+----------------------------+
|              author|               title|year|booktitle_name|booktitle_fullname|     title_processed|booktitle_name_processed|booktitle_fullname_processed|
+--------------------+--------------------+----+--------------+------------------+--------------------+------------------------+----------------------------+
|[Anish Das Sarma,...|Sailing the Infor...|2009|          CoRR|              CoRR|sailingtheinforma...|                    corr|                        corr|
|[Graham Cormode, ...|Differentially Pr...|2011|          CoRR|              CoRR|differentiallypri...|                    corr|                        corr|
|[Graham Cormode, ...|Differentially Pr...|2011|          CoRR|              CoRR|differentiallypri...|                    corr|                        corr|
|[Graham Cormode, ...|Accurate and Effi...|2012|    

- aggregate the 'booktitle_name' to array and find out the publication that needs to kick out

In [68]:
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import size
article_dropdup_agg_sources = article_dropdup.groupBy('title_processed').agg(collect_list('booktitle_name'))
article_dropdup_agg_sources = article_dropdup_agg_sources.withColumnRenamed('collect_list(booktitle_name)', 'booktitle_name_list')
article_dropdup_agg_sources.show()

+--------------------+--------------------+
|     title_processed| booktitle_name_list|
+--------------------+--------------------+
|abcoforderdepende...|           [VLDB J.]|
|accurateandeffici...|              [CoRR]|
|acompressedaccess...|[TODS, ACM Trans....|
|alaska:aflexibleb...|              [CoRR]|
|anonymizingbipart...|[Proc. VLDB Endow...|
|answeringrangeque...|[Proc. VLDB Endow...|
|approximatestring...|[FTDB, Found. Tre...|
|  bigdataintegration| [Proc. VLDB Endow.]|
|bigdataintegratio...|[IEEE Data Eng. B...|
|bottom-upevaluati...|[Theor. Comput. S...|
|certem:explaining...| [Proc. VLDB Endow.]|
|chronos:facilitat...| [Proc. VLDB Endow.]|
|class-basedgrapha...|[PVLDB, Proc. VLD...|
|combiningquantita...| [Proc. VLDB Endow.]|
|conditionalheavyh...|           [VLDB J.]|
|constraineddiffer...|              [CoRR]|
|constrainedprivat...|[IEEE Trans. Know...|
|cost-basedoptimiz...|     [Sigmod Record]|
|data-drivendomain...| [Proc. VLDB Endow.]|
|dataauditor:explo...|[PVLDB, Pr

In [69]:
CoRR_article_kick = article_dropdup_agg_sources.filter((array_contains(article_dropdup_agg_sources.booktitle_name_list, 'CoRR')) & (size(article_dropdup_agg_sources.booktitle_name_list) >= 2))

In [70]:
CoRR_article_kick.show()

+--------------------+--------------------+
|     title_processed| booktitle_name_list|
+--------------------+--------------------+
|answeringrangeque...|[Proc. VLDB Endow...|
|densesubgraphmain...|[CoRR, Proc. VLDB...|
|effectiveandcompl...|[CoRR, Proc. VLDB...|
|efficientandeffec...|     [CoRR, VLDB J.]|
|localdampening:di...|[CoRR, Proc. VLDB...|
|real-worldtraject...|[CoRR, Proc. VLDB...|
|truthfindingonthe...|[Proc. VLDB Endow...|
+--------------------+--------------------+



In [71]:
kick_title_processed = CoRR_article_kick.select('title_processed').collect()

- after dropping CoRR for article

In [72]:
kick_title_processed_list = [row['title_processed'] for row in kick_title_processed]
kick_title_processed_list

['answeringrangequeriesunderlocaldifferentialprivacy',
 'densesubgraphmaintenanceunderstreamingedgeweightupdatesforreal-timestoryidentification',
 'effectiveandcompletediscoveryoforderdependenciesviaset-basedaxiomatization',
 'efficientandeffectiveerwithprogressiveblocking',
 'localdampening:differentialprivacyfornon-numericqueriesvialocalsensitivity',
 'real-worldtrajectorysharingwithlocaldifferentialprivacy',
 'truthfindingonthedeepweb:istheproblemsolved?']

In [73]:
article_dropdup_kick_CoRR = article_dropdup.filter((article_dropdup.booktitle_name != 'CoRR') | (~article_dropdup.title_processed.isin(kick_title_processed_list)))

article_dropdup_kick_CoRR.count()

221

- inproceedings situation (no CoRR)

In [74]:
inproc_dropdup.filter(inproc_dropdup.booktitle_name == 'CoRR').show()

+------+-----+----+--------------+------------------+---------------+------------------------+----------------------------+
|author|title|year|booktitle_name|booktitle_fullname|title_processed|booktitle_name_processed|booktitle_fullname_processed|
+------+-----+----+--------------+------------------+---------------+------------------------+----------------------------+
+------+-----+----+--------------+------------------+---------------+------------------------+----------------------------+



In [75]:
inproc_dropdup_kick_CoRR = inproc_dropdup

# 5. Union the data between article and inproceedings


In [76]:
preprocess_data = article_dropdup_kick_CoRR.unionAll(inproc_dropdup_kick_CoRR)

In [77]:
preprocess_data.printSchema()

root
 |-- author: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)
 |-- booktitle_name: string (nullable = true)
 |-- booktitle_fullname: string (nullable = true)
 |-- title_processed: string (nullable = true)
 |-- booktitle_name_processed: string (nullable = true)
 |-- booktitle_fullname_processed: string (nullable = true)



In [78]:
preprocess_data.show(5)

+--------------------+--------------------+----+----------------+--------------------+--------------------+------------------------+----------------------------+
|              author|               title|year|  booktitle_name|  booktitle_fullname|     title_processed|booktitle_name_processed|booktitle_fullname_processed|
+--------------------+--------------------+----+----------------+--------------------+--------------------+------------------------+----------------------------+
|[Divesh Srivastav...|Special issue: be...|   0|             WWW|      World Wide Web|specialissue:best...|                     www|                worldwideweb|
|[Divesh Srivastav...|The calculation o...|1990|           CANDC|Computational Bio...|thecalculationoft...|                   candc|        computationalbiol...|
|[Divesh Srivastav...|Space Optimizatio...|1991|   Sigmod Record|                    |spaceoptimization...|            sigmodrecord|                            |
|[Divesh Srivastav...|Implem

# Query 2:

In [79]:
from pyspark.sql.functions import explode

- explode the dataframe according to author and select those columns not equal to 'D. S.'

In [80]:
query2_data = preprocess_data.select(explode('author').alias('author'), "title")

In [81]:
query2_data_exceptDS = query2_data.filter(query2_data.author != 'Divesh Srivastava')

In [82]:
query2_data_exceptDS.show(5)

+------------------+--------------------+
|            author|               title|
+------------------+--------------------+
|     Wook-shin Han|Special issue: be...|
|             Ge Yu|Special issue: be...|
|    David A. Micha|The calculation o...|
|  Jeffrey Naughton|Space Optimizatio...|
|Raghu Ramakrishnan|Space Optimizatio...|
+------------------+--------------------+
only showing top 5 rows



In [83]:
total_count = query2_data_exceptDS.groupby('author').count()
query2_result = total_count.orderBy(total_count['count'].desc()).limit(1)

In [84]:
query2_result.show()

+-----------+-----+
|     author|count|
+-----------+-----+
|Nick Koudas|  106|
+-----------+-----+



# Query 3:

In [85]:
query3_data = preprocess_data.select("title", "year")

In [86]:
query3_result = query3_data.groupby('year').count().orderBy(query3_data['year'].desc())

- for store the result

In [87]:
query3_result.show(200, truncate = False)

+----+-----+
|year|count|
+----+-----+
|2022|17   |
|2021|17   |
|2020|13   |
|2019|17   |
|2018|18   |
|2017|16   |
|2016|7    |
|2015|22   |
|2014|10   |
|2013|13   |
|2012|17   |
|2011|15   |
|2010|35   |
|2009|43   |
|2008|26   |
|2007|18   |
|2006|20   |
|2005|21   |
|2004|15   |
|2003|34   |
|2002|23   |
|2001|13   |
|2000|19   |
|1999|13   |
|1998|9    |
|1997|2    |
|1996|16   |
|1995|8    |
|1994|13   |
|1993|14   |
|1992|8    |
|1991|4    |
|1990|4    |
|0   |1    |
|null|1    |
+----+-----+



# Query 4:

## 1. Select 2021 publication data

In [88]:
article_2021 = df_article.filter(df_article.year == 2021).select('title', 'author', 'booktitle_name', 'year').withColumnRenamed('booktitle_name', 'journal/conference')
inproc_2021 = df_inproc.filter(df_inproc.year == 2021).select('title', 'author', 'booktitle_name', 'year').withColumnRenamed('booktitle_name', 'journal/conference')

publication_2021 = article_2021.unionAll(inproc_2021)

In [89]:
publication_2021.show(5)

+--------------------+--------------------+------------------+----+
|               title|              author|journal/conference|year|
+--------------------+--------------------+------------------+----+
|The Rule of Virtu...|[Lishan Lan, Qin ...|  Sci. Eng. Ethics|2021|
|A Multi-level Rev...|[Diana Adela Mart...|  Sci. Eng. Ethics|2021|
|Applying a Social...|[Miri Barak, Gize...|  Sci. Eng. Ethics|2021|
|Correction to: Ma...|[Filippo Santoni ...|  Sci. Eng. Ethics|2021|
|Towards a Theory ...| [Matthew J. Dennis]|  Sci. Eng. Ethics|2021|
+--------------------+--------------------+------------------+----+
only showing top 5 rows



## 2. Main Code for TF-IDF

In [90]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer

- firstly, drop all null value "title"

In [91]:
publication_2021_dropna = publication_2021.dropna(subset = 'title')

- then, calculate for TF-IDF

In [92]:
tokenizer = Tokenizer(inputCol="title", outputCol="words")
wordsData = tokenizer.transform(publication_2021_dropna)

In [93]:
wordsData.first()

Row(title='The Rule of Virtue: A Confucian Response to the Ethical Challenges of Technocracy.', author=['Lishan Lan', 'Qin Zhu', 'Yongmou Liu'], journal/conference='Sci. Eng. Ethics', year=2021, words=['the', 'rule', 'of', 'virtue:', 'a', 'confucian', 'response', 'to', 'the', 'ethical', 'challenges', 'of', 'technocracy.'])

In [94]:
wordsData = wordsData.filter(array_contains(wordsData.words, 'self-attention') | array_contains(wordsData.words, 'transformer'))

In [95]:
wordsData.first()

Row(title='Application of solely self-attention mechanism in CSI-fingerprinting-based indoor localization.', author=['Kabo Poloko Nkabiti', 'Yueyun Chen'], journal/conference='Neural Comput. Appl.', year=2021, words=['application', 'of', 'solely', 'self-attention', 'mechanism', 'in', 'csi-fingerprinting-based', 'indoor', 'localization.'])

In [96]:
# TF-IDF computation
countVect = CountVectorizer(inputCol='words', outputCol='vocabularys')
model = countVect.fit(wordsData)
featurizedData = model.transform(wordsData)


idf = IDF(inputCol="vocabularys", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [97]:
rescaledData.first()

Row(title='Application of solely self-attention mechanism in CSI-fingerprinting-based indoor localization.', author=['Kabo Poloko Nkabiti', 'Yueyun Chen'], journal/conference='Neural Comput. Appl.', year=2021, words=['application', 'of', 'solely', 'self-attention', 'mechanism', 'in', 'csi-fingerprinting-based', 'indoor', 'localization.'], vocabularys=SparseVector(3802, {3: 1.0, 6: 1.0, 8: 1.0, 84: 1.0, 141: 1.0, 344: 1.0, 661: 1.0, 2860: 1.0, 3227: 1.0}), features=SparseVector(3802, {3: 1.6129, 6: 1.892, 8: 2.0474, 84: 4.3769, 141: 4.6954, 344: 5.3885, 661: 6.0816, 2860: 6.7748, 3227: 6.7748}))

In [98]:
from pyspark.sql.functions import udf, col

- the index of 'self-attention' and 'transformer'

In [99]:
keywords_index = [model.vocabulary.index(word) for word in ['self-attention', 'transformer']]
keywords_index

[3, 0]

In [100]:
values_udf = udf(lambda vector: vector.toArray().tolist(),ArrayType(DoubleType()))

rescaledData = rescaledData.withColumn('TF-IDF_list', values_udf(col('features')))
rescaledData.first()

Row(title='Application of solely self-attention mechanism in CSI-fingerprinting-based indoor localization.', author=['Kabo Poloko Nkabiti', 'Yueyun Chen'], journal/conference='Neural Comput. Appl.', year=2021, words=['application', 'of', 'solely', 'self-attention', 'mechanism', 'in', 'csi-fingerprinting-based', 'indoor', 'localization.'], vocabularys=SparseVector(3802, {3: 1.0, 6: 1.0, 8: 1.0, 84: 1.0, 141: 1.0, 344: 1.0, 661: 1.0, 2860: 1.0, 3227: 1.0}), features=SparseVector(3802, {3: 1.6129, 6: 1.892, 8: 2.0474, 84: 4.3769, 141: 4.6954, 344: 5.3885, 661: 6.0816, 2860: 6.7748, 3227: 6.7748}), TF-IDF_list=[0.0, 0.0, 0.0, 1.6128704100834248, 0.0, 0.0, 1.8919932291395358, 0.0, 2.047407333013566, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [101]:
sum_keyword = udf(lambda vector: vector[0] + vector[3], FloatType())

rescaledData = rescaledData.withColumn('TF-IDF_keyword', sum_keyword(col('TF-IDF_list')))
rescaledData.select('title', 'author', 'journal/conference', 'TF-IDF_keyword').first()

Row(title='Application of solely self-attention mechanism in CSI-fingerprinting-based indoor localization.', author=['Kabo Poloko Nkabiti', 'Yueyun Chen'], journal/conference='Neural Comput. Appl.', TF-IDF_keyword=1.612870454788208)

In [102]:
TFIDF_Data = rescaledData.select('title', 'author', 'journal/conference', 'year', 'TF-IDF_keyword')
TFIDF_Data.show()

+--------------------+--------------------+--------------------+----+--------------+
|               title|              author|  journal/conference|year|TF-IDF_keyword|
+--------------------+--------------------+--------------------+----+--------------+
|Application of so...|[Kabo Poloko Nkab...|Neural Comput. Appl.|2021|     1.6128705|
|Transformer guide...|[Xiangyu Li, Yong...|Neural Comput. Appl.|2021|    0.21376449|
|Joint extraction ...|[Zhao Meng, Sheng...|J. Exp. Theor. Ar...|2021|     1.6128705|
|Data augmentation...|[Ibrahim Saad Aly...|  Expert Syst. Appl.|2021|     1.6128705|
|An end-to-end fra...|[Can-can Jin, Xi ...|  Expert Syst. Appl.|2021|    0.21376449|
|Improved few-shot...|[Yaoyu Xu, Yuan L...|  Expert Syst. Appl.|2021|    0.21376449|
|Candidate point s...|[Hyeonuk Kim, Kyu...|  Expert Syst. Appl.|2021|     1.6128705|
|Detection of tube...|[Linh T. Duong, N...|  Expert Syst. Appl.|2021|    0.21376449|
|Local-constraint ...|       [Jincheng Hu]|Int. J. Comput. S...|2

In [103]:
query4_result = TFIDF_Data.orderBy(TFIDF_Data['TF-IDF_keyword'].desc()).select('title', 'author', 'journal/conference', 'year').limit(10)

query4_result.show(10)

+--------------------+--------------------+--------------------+----+
|               title|              author|  journal/conference|year|
+--------------------+--------------------+--------------------+----+
|Spatial self-atte...|[Adu Asare Baffou...|J. Vis. Commun. I...|2021|
|Synthesizer: Reth...|[Yi Tay, Dara Bah...|                ICML|2021|
|Shatter: An Effic...|[Ran Tian, Joshua...|                CoRR|2021|
|Multi-View Self-A...|[Rui Wang, Junyi ...|                CoRR|2021|
|Multi-Head Self-A...|[Faisal Alamri, A...|                CoRR|2021|
|A Vision Transfor...|[Yuxiang Guo, Di ...|      ICISCAE (IEEE)|2021|
|COViT-GAN: Vision...|[Ara Abigail E. A...|           ICANN (2)|2021|
|E.T.: re-thinking...|[Shiyang Chen, Sh...|                  SC|2021|
|Dual Aspect Self-...|[Zhizheng Zhang 0...|                CoRR|2021|
|EEG-Transformer: ...|[Young Eun Lee, S...|                CoRR|2021|
+--------------------+--------------------+--------------------+----+



- for store the result

In [104]:
query4_result.show(10, truncate = False)

+-----------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+--------------------------------+----+
|title                                                                                                                  |author                                                                                                              |journal/conference              |year|
+-----------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+--------------------------------+----+
|Spatial self-attention network with self-attention distillation for fine-grained image recognition.                    |[Adu Asare Baffour, Zhen Qin 0002, Yong Wang, Zh